In [1]:
%load_ext watermark
%watermark -a 'Christian Schuhegger' -u -d -v -p numpy,scipy,pandas,matplotlib,seaborn,tensorflow

Christian Schuhegger 
last updated: 2019-02-26 

CPython 3.6.8
IPython 7.3.0

numpy 1.16.1
scipy 1.2.0
pandas 0.24.1
matplotlib 3.0.2
seaborn 0.9.0
tensorflow 1.12.0


In [2]:
%matplotlib inline
import numpy as np, scipy, scipy.stats as stats, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import sklearn, sklearn.pipeline, sklearn.model_selection, sklearn.preprocessing
import logging, time, datetime
import tensorflow as tf, tensorflow.keras as keras


pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(linewidth=1000)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

sns.set()

In [3]:
tf.keras.__version__

'2.1.6-tf'

In [4]:
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        if type(df) == np.ndarray:
            df = pd.DataFrame(df)
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline"')
    # print(html_str)
    display_html(html_str,raw=True)

CSS = """
.output {
    flex-direction: row;
}
"""

def display_graphs_side_by_side(*args):
    html_str='<table><tr>'
    for g in args:
        html_str += '<td>'
        html_str += g._repr_svg_()
        html_str += '</td>'
    html_str += '</tr></table>'
    display_html(html_str,raw=True)
    

display(HTML("<style>.container { width:70% !important; }</style>"))

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3712538308586960821
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10848146955848524755
physical_device_desc: "device: XLA_CPU device"
]


In [6]:
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s:%(name)s:%(levelname)s: %(message)s')
# log = logging.getLogger('std')

In [7]:
N_in       = 1000
N_subjects = 260 * 10 # 100000
N_subjects = 100000
W = stats.norm(loc=0, scale=1).rvs(size=(2,N_in), random_state=np.random.RandomState(42))
X = stats.norm(loc=0, scale=1).rvs(size=(N_subjects,N_in), random_state=np.random.RandomState(43))
y = np.sum(W[1:,:] * X + W[0,:], axis=1)

In [8]:
y.shape, X.shape

((100000,), (100000, 1000))

In [9]:
pd.Series(y).describe()

count    100000.000000
mean         19.234805
std          31.516783
min        -147.380827
25%          -2.102370
50%          19.273695
75%          40.475011
max         147.253509
dtype: float64

In [10]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.1, random_state = 42)

In [11]:
batch_size = 256
epochs     = 20

In [12]:
def create_tensorflow_keras_model(input_tensor):
    ACTIVATION = 'relu'

    net = input_tensor

    net = keras.layers.Dense(300, activation=ACTIVATION, name='fc-1')(net)
    net = keras.layers.Dense(100, activation=ACTIVATION, name='fc-2')(net)
    net = keras.layers.Dense(1, activation=None, name='predictions')(net)

    return net

In [13]:
sym_X_in  = keras.layers.Input(shape=[X.shape[1]])
sym_y_out = create_tensorflow_keras_model(sym_X_in)

tf_keras_model = keras.models.Model(sym_X_in, sym_y_out, name='MLP')
tf_keras_model.compile("adam", "mean_squared_error", ["mean_absolute_error"])

time1 = time.time()
tf_keras_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)#, validation_split=0.1
time2 = time.time()

Epoch 1/20
90000/90000 [==============================] - 2s 23us/step - loss: 158.0763 - mean_absolute_error: 6.3414
Epoch 2/20
90000/90000 [==============================] - 2s 21us/step - loss: 2.9508 - mean_absolute_error: 1.3592
Epoch 3/20
90000/90000 [==============================] - 2s 20us/step - loss: 1.6632 - mean_absolute_error: 1.0218
Epoch 4/20
90000/90000 [==============================] - 2s 21us/step - loss: 1.0834 - mean_absolute_error: 0.8221
Epoch 5/20
90000/90000 [==============================] - 2s 20us/step - loss: 0.8003 - mean_absolute_error: 0.7032
Epoch 6/20
90000/90000 [==============================] - 2s 21us/step - loss: 0.6689 - mean_absolute_error: 0.6428
Epoch 7/20
90000/90000 [==============================] - 2s 21us/step - loss: 0.6334 - mean_absolute_error: 0.6251
Epoch 8/20
90000/90000 [==============================] - 2s 21us/step - loss: 0.7128 - mean_absolute_error: 0.6655
Epoch 9/20
90000/90000 [==============================] - 2s 21us/step

In [14]:
print('time: {}'.format(time2-time1))

time: 37.65403413772583


In [15]:
y_tf_keras_pred = tf_keras_model.predict(X_test)
s = sklearn.metrics.mean_squared_error(y_test, y_tf_keras_pred)
s

1.4023710775244795

In [16]:
sklearn.metrics.explained_variance_score(y_test, y_tf_keras_pred)

0.9985703939215318